# AI Project Code - Viterbi Algorithm for POS Tagging

## Group Members :  
18ucc002 : Deepesh Kanodia  
18ucc058 : Naman Maheshwari  
18ucc160 : Manav Ladha  
18ucs179 : Shreena Parekh  




The entire code is majorly divided into 5 segments:

1. **Importing Necessaries**   
    1.1 Importing Libraries  
    1.2 Importing Dataset
  
2. **Data Preprocessing**   
    2.1 Extracting from .txt file   
    2.2 Removing empty list  
    2.3 Converting to Tuple  
    2.4 Unique Tags  
    2.5 Unique Words - Vocabulary  
  
3. **Computing Probabilities**  
    3.1 Map Tag to Index     
    3.2 Compute Word to Tag Count Matrix   
    3.3 Compute Tag to Tag Count Matrix    
    3.4 Compute Emission Probability Matrix   
    3.5 Compute Transition Probability Matrix   
  
4. **Viterbi Algorithm**   
    4.1 Viterbi Decoding Function  
    4.2 Backtracking Function  
    4.3 Map output state sequence   
  
5. **Testing**   
    5.1 Test code on an exapmle  
    5.2 Test code on entire testing set  


## 1.1 Importing Libraries

In [1]:
import nltk
import numpy as np
import pandas as pd
import random
import pprint, time
from collections import defaultdict
import statistics 
from scipy import stats

## 1.2 Importing Datasets

In [2]:
test_file = open("test.txt", "r")
train_file = open("train.txt", "r")

## 2.1 Extracting from .txt

In [3]:
train_list_of_lists = []
for line in train_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    train_list_of_lists.append(line_list)

train_file.close()

In [4]:
train_list_of_lists[30:50]
print("Number of Training Examples : ",len(train_list_of_lists))

Number of Training Examples :  220663


In [5]:
test_list_of_lists = []
for line in test_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    test_list_of_lists.append(line_list)

test_file.close()

In [6]:
test_list_of_lists[30:50]
print("Number of Testing Examples : ",len(test_list_of_lists))

Number of Testing Examples :  49389


## 2.2 Removing empty list

In [7]:

train_list_of_lists = [ele for ele in train_list_of_lists if ele!=[]]
print("Number of Training Examples after removing empty lists : ",len(train_list_of_lists))


Number of Training Examples after removing empty lists :  211727


## 2.3 Converting to Tuple

In [8]:
final_train_list_pos =[]
for i in train_list_of_lists :
    k = i[:2]
    j = tuple(k)
    final_train_list_pos.append(j)

In [9]:
final_test_list_pos =[]
for i in test_list_of_lists :
    k = i[:2]
    j = tuple(k)
    final_test_list_pos.append(j)

In [10]:
print(len(final_train_list_pos))
final_train_list_pos[:10]

211727


[('Confidence', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('pound', 'NN'),
 ('is', 'VBZ'),
 ('widely', 'RB'),
 ('expected', 'VBN'),
 ('to', 'TO'),
 ('take', 'VB'),
 ('another', 'DT')]

In [11]:
print(len(final_test_list_pos))
final_test_list_pos[:10]

49389


[('Rockwell', 'NNP'),
 ('International', 'NNP'),
 ('Corp.', 'NNP'),
 ("'s", 'POS'),
 ('Tulsa', 'NNP'),
 ('unit', 'NN'),
 ('said', 'VBD'),
 ('it', 'PRP'),
 ('signed', 'VBD'),
 ('a', 'DT')]

## 2.4 Unique POS tags 

In [12]:
unique_pos_tags = list({tag for word,tag in final_train_list_pos})
print(unique_pos_tags)

['VB', ')', 'RBS', 'IN', 'JJ', 'WDT', 'WP$', 'SYM', 'PDT', 'JJR', 'EX', 'UH', 'FW', ',', '(', 'DT', 'POS', 'WP', 'VBP', 'MD', 'PRP', 'JJS', 'RP', 'VBN', 'NN', '.', 'NNP', 'TO', 'CC', '``', 'NNPS', 'RBR', '$', 'WRB', ':', 'PRP$', 'RB', "''", 'VBZ', 'NNS', 'CD', 'VBD', 'VBG', '#']


In [13]:
print("Number of Unique POS Tag in Training Set : ",len(unique_pos_tags))

Number of Unique POS Tag in Training Set :  44


## 2.5 Unique Words - Vocabulary

In [14]:
# check total words in vocabulary
vocab = {word for word,tag in final_train_list_pos}
print("Number of Unique Words in Training Set : ",len(vocab))


Number of Unique Words in Training Set :  19122


## 3.1 Map index to Tag

In [15]:
# Function to map the state tags to unique indices i.e. 0 : NN , 1 : DT

def assign_index_tag(unique_pos_tags):
    tag_map={}
    vals = list(range(0,len(unique_pos_tags)))
    
    for i , tag in enumerate(unique_pos_tags):
        #tag_map[tag] = i
        tag_map[i] = tag
        
    #tag_map[unique_pos_tags]=vals
    return tag_map

In [16]:
tag_map = assign_index_tag(unique_pos_tags)
print(tag_map)

{0: 'VB', 1: ')', 2: 'RBS', 3: 'IN', 4: 'JJ', 5: 'WDT', 6: 'WP$', 7: 'SYM', 8: 'PDT', 9: 'JJR', 10: 'EX', 11: 'UH', 12: 'FW', 13: ',', 14: '(', 15: 'DT', 16: 'POS', 17: 'WP', 18: 'VBP', 19: 'MD', 20: 'PRP', 21: 'JJS', 22: 'RP', 23: 'VBN', 24: 'NN', 25: '.', 26: 'NNP', 27: 'TO', 28: 'CC', 29: '``', 30: 'NNPS', 31: 'RBR', 32: '$', 33: 'WRB', 34: ':', 35: 'PRP$', 36: 'RB', 37: "''", 38: 'VBZ', 39: 'NNS', 40: 'CD', 41: 'VBD', 42: 'VBG', 43: '#'}


## 3.2 Compute Word to Tag Count Matrix

In [17]:
# Function to compute the word to tag count matrix which stores the count of each word w being in each state t

def compute_count_matrix_wordtotag(vocab,unique_pos_tags,train_set):
    
    count_mat_wt = pd.DataFrame(0,columns = list(vocab), index=list(unique_pos_tags))
    count_mat_t = pd.DataFrame(0,columns = ['count'], index=list(unique_pos_tags))
    
    for w,t in train_set :
        count_mat_t.loc[t,'count']+=1
        count_mat_wt.loc[t,w]+=1  
  
    return count_mat_t,count_mat_wt
        
        
    

In [18]:
count_mat_t,count_mat_wt = compute_count_matrix_wordtotag(vocab,unique_pos_tags,final_train_list_pos)

In [19]:
print("WORD to TAG COUNT MATRIX")
count_mat_wt

WORD to TAG COUNT MATRIX


,owns,grassroots,propelled,brought,kept,deputies,Wisconsin,stem,barreling,Sibra,...,responsibilities,president-elect,reflect,tasteless,borrower,worries,20-mile,prod,due,3.34
VB,0,0,0,0,0,0,0,3,0,0,...,0,0,10,0,0,0,0,1,0,0
),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RBS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JJ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,87,0
WDT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP$,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SYM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PDT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JJR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
print("TAG COUNT MATRIX")
count_mat_t

TAG COUNT MATRIX


,count
VB,6017
),281
RBS,191
IN,22764
JJ,13085
WDT,955
WP$,35
SYM,6
PDT,55
JJR,853


## 3.3 Compute Tag to Tag Count Matrix

In [21]:
# Function to compute the tag to tag count matrix which stores the count of each tag t1 being preceded by each state t2

def compute_count_matrix_tagtotag(vocab,unique_pos_tags,train_set):
    
    count_mat_tt = pd.DataFrame(0,columns = list(unique_pos_tags), index=list(unique_pos_tags))
    
    for i in range(0,len(train_set)-1) :
        cur_tag = train_set[i][1]
        nxt_tag = train_set[i+1][1]
        count_mat_tt.loc[cur_tag,nxt_tag]+=1   
  
    return count_mat_tt
                

In [23]:
count_mat_tt = compute_count_matrix_tagtotag(vocab,unique_pos_tags,final_train_list_pos)

In [24]:
print("TAG to TAG COUNT MATRIX")
count_mat_tt

TAG to TAG COUNT MATRIX


,VB,),RBS,IN,JJ,WDT,WP$,SYM,PDT,JJR,...,:,PRP$,RB,'',VBZ,NNS,CD,VBD,VBG,#
VB,41,0,5,841,496,6,0,0,5,81,...,11,292,274,21,15,351,125,8,98,1
),3,0,0,36,6,0,0,0,0,0,...,23,0,5,0,19,7,2,8,0,0
RBS,1,0,0,37,80,0,0,0,0,0,...,0,0,20,0,0,26,0,0,1,0
IN,10,1,41,677,1916,74,0,0,28,122,...,10,835,341,12,30,1401,1414,4,631,16
JJ,5,7,2,703,959,7,1,0,1,2,...,36,4,69,59,9,2945,202,20,28,1
WDT,0,0,0,9,5,0,0,0,0,0,...,1,3,27,0,269,17,3,246,0,0
WP$,0,0,0,0,5,0,0,0,0,1,...,0,0,0,0,0,10,0,0,1,0
SYM,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PDT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JJR,8,1,0,307,22,0,0,0,0,0,...,3,2,1,0,1,122,3,0,6,0


## 3.4 Compute Emission Probability Matrix

In [25]:
# Function to compute the emission probability matrix using the word to tag count matrix and tag count matrix

def compute_emission_matrix(count_mat_t,count_mat_wt):
    emission_mat = pd.DataFrame(0,columns = list(vocab), index=list(unique_pos_tags))
    
    for t in unique_pos_tags:
        for w in vocab:
            emission_mat.loc[t,w]=count_mat_wt.loc[t,w]/count_mat_t.loc[t,'count']
    
    return emission_mat
    


In [26]:
emmision_mat = compute_emission_matrix(count_mat_t,count_mat_wt)

In [27]:
print("EMISSION MATRIX")
emmision_mat

EMISSION MATRIX


,owns,grassroots,propelled,brought,kept,deputies,Wisconsin,stem,barreling,Sibra,...,responsibilities,president-elect,reflect,tasteless,borrower,worries,20-mile,prod,due,3.34
VB,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000499,0.000000,0.000000,...,0.00000,0.000000,0.001662,0.000000,0.000000,0.000000,0.000000,0.000166,0.000000,0.00000
),0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
RBS,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
IN,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
JJ,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000076,0.000000,0.000000,0.000076,0.000000,0.006649,0.00000
WDT,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
WP$,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
SYM,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
PDT,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
JJR,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [28]:
emmision_mat.loc['NNP','September']

0.0026654596660631662

## 3.5 Compute Transition Probability Matrix

In [29]:
# Function to compute the transition probability matrix using the tag to tag count matrix and tag count matrix

def compute_transition_matrixx(count_mat_t,count_mat_tt):
    
    transition_mat = pd.DataFrame(0,columns = list(unique_pos_tags), index=list(unique_pos_tags))
    
    for t1 in unique_pos_tags:
        for t2 in unique_pos_tags:
             transition_mat.loc[t1,t2]=count_mat_tt.loc[t1,t2]/count_mat_t.loc[t1,'count']
    
    return  transition_mat
    

In [30]:
transition_mat = compute_transition_matrixx(count_mat_t,count_mat_tt)

In [31]:
transition_mat

,VB,),RBS,IN,JJ,WDT,WP$,SYM,PDT,JJR,...,:,PRP$,RB,'',VBZ,NNS,CD,VBD,VBG,#
VB,0.006814,0.000000,0.000831,0.139771,0.082433,0.000997,0.000000,0.000000,0.000831,0.013462,...,0.001828,0.048529,0.045538,0.003490,0.002493,0.058335,0.020774,0.001330,0.016287,0.000166
),0.010676,0.000000,0.000000,0.128114,0.021352,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.081851,0.000000,0.017794,0.000000,0.067616,0.024911,0.007117,0.028470,0.000000,0.000000
RBS,0.005236,0.000000,0.000000,0.193717,0.418848,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.104712,0.000000,0.000000,0.136126,0.000000,0.000000,0.005236,0.000000
IN,0.000439,0.000044,0.001801,0.029740,0.084168,0.003251,0.000000,0.000000,0.001230,0.005359,...,0.000439,0.036681,0.014980,0.000527,0.001318,0.061545,0.062116,0.000176,0.027719,0.000703
JJ,0.000382,0.000535,0.000153,0.053726,0.073290,0.000535,0.000076,0.000000,0.000076,0.000153,...,0.002751,0.000306,0.005273,0.004509,0.000688,0.225067,0.015438,0.001528,0.002140,0.000076
WDT,0.000000,0.000000,0.000000,0.009424,0.005236,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001047,0.003141,0.028272,0.000000,0.281675,0.017801,0.003141,0.257592,0.000000,0.000000
WP$,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.028571,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.028571,0.000000
SYM,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PDT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
JJR,0.009379,0.001172,0.000000,0.359906,0.025791,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003517,0.002345,0.001172,0.000000,0.001172,0.143025,0.003517,0.000000,0.007034,0.000000


In [32]:
print(transition_mat['POS']['NNS'])

0.009839195241941405


## 4.1 Viterbi Algorithm

In [33]:
#Function to compute the trelis matrix and backpointer matrix to keep track of maximum probablity value and path

def Viterbi (tag_map , emmision_mat , transition_mat , init_dist , end_dist , observation):    
        
    N = transition_mat.shape[0]
    T = len(observation)
    
    rows = N + 2
    cols = T + 1
     
    trelis = np.zeros((rows, cols),dtype='float64')    
    backpointer = np.zeros((rows, cols),dtype='float64')
    start_symbol = observation[0]
    
    transition_matrix = transition_mat.to_numpy()
    if start_symbol not in emmision_mat.columns:
            emmision_mat[start_symbol] = 0.00001
        
    #INITIALIZATOIN
   
    for s in range(0,N):
        
        trelis[s][0] = init_dist[s] * emmision_mat.loc[tag_map[s] ,start_symbol] 
        backpointer[s][0] = 0
        
        
    #TRAVERSING
    
    for t in range(1,T):
        
        cur_obs = observation[t]
        if cur_obs not in emmision_mat.columns:
            emmision_mat[cur_obs]=0.00001
           
        for s in range(0,N):
           
            max_val = -1
            max_pos = -1
            for pre_s in range(0,N):
                pre_viterbi = trelis[pre_s][t-1]
                transition_val = transition_matrix[pre_s][s]
                
                val = pre_viterbi * transition_val
                
                if val>max_val:
                    max_val = val
                    max_pos = pre_s
                    
            trelis[s][t] = max_val * emmision_mat.loc[tag_map[s],cur_obs]
            backpointer[s][t] = max_pos
            
   
     
    #TERMINATING
    
    max_end_value = -1
    max_end_position = -1
    for s in range(0,N):
        
        end_viterbi = trelis[s][T-1]
        end_transition_val = end_dist[s]
        
        end_value = end_viterbi * end_transition_val
        
        if end_value > max_end_value:
            max_end_value = end_value
            max_end_pos = s
            
    trelis[N][T] = max_end_value
    backpointer[N][T] = max_end_pos
    
    return trelis , backpointer

## 4.2 Backtrace Function

In [34]:
#Function to find the optimal sequence of state indices from the Backpointer matrix

def backtrace_seq(backpointer):

    last_state = backpointer[-2][-1]
    back_df = pd.DataFrame(data = backpointer)  

    max_occur_list = []
    
    for col in back_df.columns[1:-1]:
        
        max_occur = int(stats.mode(list(back_df[col]))[0])
        max_occur_list.append(max_occur)
   
    
    max_occur_list.append(int(last_state))
    return max_occur_list

## 4.3 Map Output Sequence

In [35]:
#Function to map the sequence of state indices to actual state tag names

def index_output_seq(output_seq , tag_map):
    optimal_states = []
    for index in output_seq:
        optimal_states.append(tag_map[index])
        
    return optimal_states

## 5.1 Testing Algorithm on an Example

In [37]:
#Setting initial parameteres like number of states , initial probability distribution and final probability distribution

N = transition_mat.shape[0]
init_dist = list(transition_mat.loc['.',:])
end_dist = list(transition_mat['.'])

observation = ['I','have','a','dog']
original_result = ['PRP','VBP','DT','NN']

In [39]:
start = time.time()
trelis , backpointer = Viterbi (tag_map , emmision_mat , transition_mat , init_dist , end_dist , observation)
max_occur_list = backtrace_seq(backpointer)
output_state_seq = index_output_seq(max_occur_list , tag_map)
end = time.time()

In [40]:
max_occur_list

[20, 18, 15, 24]

In [41]:
output_state_seq

['PRP', 'VBP', 'DT', 'NN']

In [42]:
correctly_labeled = 0
for i in range(len(observation)):
    if output_state_seq[i]==original_result[i]:
        correctly_labeled+=1
print("ACCURACY OF THE ALGORITHM : ",correctly_labeled/len(observation)*100,"%")
print("TIME TAKEN : ",end-start)

ACCURACY OF THE ALGORITHM :  100.0 %
TIME TAKEN :  0.01622486114501953


## 5.2 Testing Algorithm on entire dataset

In [43]:
#Function to preprocess the test dataset into list of sentences to be fed to the Viterbi Algorithm

def generate_test_data(test_set):

    test_sentences=[]
    test_original_outputs=[]
    sentence = []
    output = []
    for test_case in test_set:
        if test_case:
            sentence.append(test_case[0])
            output.append(test_case[1])
        else:
            
            test_sentences.append(sentence)
            test_original_outputs.append(output)
            sentence=[]
            output=[]
    return test_sentences , test_original_outputs



In [44]:
test_sentences , test_original_outputs = generate_test_data(final_test_list_pos)
print("Number of test sentences : ",len(test_sentences))

Number of test sentences :  2012


In [ ]:
"""
For each of the test sentence we print :
    1. x - The input observation
    2. y - The original output state sequence
    3. y_pred - The predicted output state sequence
"""
 
i=1
for x,y in zip(test_sentences , test_original_outputs):
    print("------------TEST CASE ",i," ------------------")
    print("Input x : ",x)
    print("Original Output y : ",y)
    
    trelis , backpointer = Viterbi (tag_map , emmision_mat , transition_mat , init_dist , end_dist , x)
    max_occur_list = backtrace_seq(backpointer)
    y_pred = index_output_seq(max_occur_list , tag_map)

    print("Predicted Output y_pred : ",y_pred)
    print()
    i+=1

------------TEST CASE  1  ------------------
Input x :  ['Rockwell', 'International', 'Corp.', "'s", 'Tulsa', 'unit', 'said', 'it', 'signed', 'a', 'tentative', 'agreement', 'extending', 'its', 'contract', 'with', 'Boeing', 'Co.', 'to', 'provide', 'structural', 'parts', 'for', 'Boeing', "'s", '747', 'jetliners', '.']
Original Output y :  ['NNP', 'NNP', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'VBG', 'PRP$', 'NN', 'IN', 'NNP', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CD', 'NNS', '.']
Predicted Output y_pred :  ['NNP', 'NNP', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'IN', 'VB', 'NN', 'IN', 'NNP', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CD', ',', '.']

------------TEST CASE  2  ------------------
Input x :  ['Rockwell', 'said', 'the', 'agreement', 'calls', 'for', 'it', 'to', 'supply', '200', 'additional', 'so-called', 'shipsets', 'for', 'the', 'planes', '.']
Original Output y :  ['NNP', 'VBD', 'DT', 'NN', 'VBZ', 'IN

Predicted Output y_pred :  ['IN', 'DT', 'CD', 'NNS', ',', 'PRP', 'VBD', 'DT', 'NN', 'IN', 'VB', 'CD', 'CD', ',', 'CC', 'VB', 'CD', 'DT', 'NN', ',', 'IN', 'NNS', 'IN', 'VB', 'CD', 'CD', ',', 'CC', 'VB', 'CD', 'DT', 'NN', ',', 'IN', 'DT', 'CD', 'NN', '.']

------------TEST CASE  19  ------------------
Input x :  ['Great', 'American', 'said', 'it', 'increased', 'its', 'loan-loss', 'reserves', 'by', '$', '93', 'million', 'after', 'reviewing', 'its', 'loan', 'portfolio', ',', 'raising', 'its', 'total', 'loan', 'and', 'real', 'estate', 'reserves', 'to', '$', '217', 'million', '.']
Original Output y :  ['NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'PRP$', 'NN', 'NNS', 'IN', '$', 'CD', 'CD', 'IN', 'VBG', 'PRP$', 'NN', 'NN', ',', 'VBG', 'PRP$', 'JJ', 'NN', 'CC', 'JJ', 'NN', 'NNS', 'TO', '$', 'CD', 'CD', '.']
Predicted Output y_pred :  ['NNP', 'NNP', 'VBD', 'PRP', 'VBD', 'VB', 'NN', 'NNS', 'IN', 'VB', 'CD', 'CD', 'IN', 'VBG', 'VB', 'NN', 'NN', ',', 'VBG', 'VB', 'NN', 'NN', 'CC', 'JJ', 'NN', 'NNS', 'TO', '

Predicted Output y_pred :  ['NNP', 'NNP', 'VBZ', 'CD', 'NNP', 'NNS', 'CC', 'VBZ', 'NNS', 'TO', 'VB', 'DT', 'CD', 'IN', 'VB', 'CD', 'DT', '.']

------------TEST CASE  30  ------------------
Input x :  ['In', 'the', '$', '300-a-share', 'buyout', ',', 'that', 'totaled', 'about', '$', '76.7', 'million', '.']
Original Output y :  ['IN', 'DT', '$', 'JJ', 'NN', ',', 'WDT', 'VBD', 'RB', '$', 'CD', 'CD', '.']
Predicted Output y_pred :  ['IN', 'DT', 'VB', 'JJ', 'NN', ',', 'IN', 'VBD', 'IN', 'VB', 'CD', 'CD', '.']

------------TEST CASE  31  ------------------
Input x :  ['By', 'yesterday', "'s", 'close', 'of', 'trading', ',', 'it', 'was', 'good', 'for', 'a', 'paltry', '$', '43.5', 'million', '.']
Original Output y :  ['IN', 'NN', 'POS', 'NN', 'IN', 'NN', ',', 'PRP', 'VBD', 'JJ', 'IN', 'DT', 'JJ', '$', 'CD', 'CD', '.']
Predicted Output y_pred :  ['IN', 'NN', 'POS', 'NN', 'IN', 'NN', ',', 'PRP', 'VBD', 'JJ', 'IN', 'DT', 'NN', 'VB', 'CD', 'CD', '.']

------------TEST CASE  32  ------------------
In

Predicted Output y_pred :  ['NNP', 'NNP', 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'VB', 'VBD', 'TO', 'VB', 'JJ', 'NNS', ',', 'IN', 'DT', 'NNS', '.']

------------TEST CASE  44  ------------------
Input x :  ['Manville', 'is', 'a', 'building', 'and', 'forest', 'products', 'concern', '.']
Original Output y :  ['NNP', 'VBZ', 'DT', 'NN', 'CC', 'NN', 'NNS', 'VBP', '.']
Predicted Output y_pred :  ['NNP', 'VBZ', 'DT', 'NN', 'CC', 'NN', 'NNS', 'VBP', '.']

------------TEST CASE  45  ------------------
Input x :  ['US', 'Facilities', 'Corp.', 'said', 'Robert', 'J.', 'Percival', 'agreed', 'to', 'step', 'down', 'as', 'vice', 'chairman', 'of', 'the', 'insurance', 'holding', 'company', '.']
Original Output y :  ['PRP', 'NNP', 'NNP', 'VBD', 'NNP', 'NNP', 'NNP', 'VBD', 'TO', 'VB', 'RB', 'IN', 'NN', 'NN', 'IN', 'DT', 'NN', 'VBG', 'NN', '.']
Predicted Output y_pred :  ['PRP', 'VBD', 'NNP', 'VBD', 'NNP', 'NNP', 'NNP', 'VBD', 'TO', 'VB', 'RB', 'IN', 'NN', 'NN', 'IN', 'DT', 'NN', 'VBG', 'NN', '.']

------------T

Predicted Output y_pred :  ['NNP', ',', 'NNP', 'POS', 'JJ', 'NN', 'NN', ',', 'VBD', 'JJR', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'RB', 'VBN', 'IN', 'DT', 'NNP', 'IN', 'DT', 'NN', 'VBG', 'NN', ',', 'DT', 'NN', 'VBD', '.']

------------TEST CASE  64  ------------------
Input x :  ['The', 'drug', 'was', 'introduced', 'in', 'West', 'Germany', 'this', 'year', '.']
Original Output y :  ['DT', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'NNP', 'DT', 'NN', '.']
Predicted Output y_pred :  ['DT', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'NNP', 'DT', 'NN', '.']

------------TEST CASE  65  ------------------
Input x :  ['Intense', 'competition', ',', 'however', ',', 'led', 'to', 'unit', 'sales', 'declines', 'for', 'a', 'group', 'of', 'Merck', "'s", 'established', 'human', 'and', 'animal-health', 'products', ',', 'including', 'Aldomet', 'and', 'Indocin', '.']
Original Output y :  ['JJ', 'NN', ',', 'RB', ',', 'VBN', 'TO', 'NN', 'NNS', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNP', 'POS', 'VBN', 'NN', 'CC', 'NN', 'NNS', '

...

Predicted Output y_pred :  ['``', 'PRP', 'NN', 'VB', 'JJ', 'NNS', 'TO', 'VB', 'DT', ',', "''", 'VBD', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', '.']

------------TEST CASE  1706  ------------------
Input x :  ['``', 'We', 'do', "n't", 'want', 'to', 'bring', 'back', 'something', 'the', 'members', 'will', 'reject', '.', "''"]
Original Output y :  ['``', 'PRP', 'VBP', 'RB', 'VB', 'TO', 'VB', 'JJ', 'NN', 'DT', 'NNS', 'MD', 'VB', '.', "''"]
Predicted Output y_pred :  ['``', 'PRP', 'VBP', 'RB', 'VB', 'TO', 'VB', 'RB', 'NN', 'DT', 'NNS', 'NN', 'VB', '.', "''"]

------------TEST CASE  1707  ------------------
Input x :  ['Machinists', 'already', 'have', 'rejected', 'a', 'package', 'that', 'would', 'have', 'provided', 'a', '10', '%', 'pay', 'raise', 'plus', 'bonuses', 'over', 'the', 'three-year', 'life', 'of', 'the', 'contract', '.']
Original Output y :  ['NNS', 'RB', 'VBP', 'VBN', 'DT', 'NN', 'WDT', 'MD', 'VB', 'VBN', 'DT', 'CD', 'NN', 'NN', 'NN', 'CC', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN

Predicted Output y_pred :  ['``', 'IN', 'VBG', 'JJ', ',', 'JJ', ',', 'NNP', 'NN', 'PRP', 'VB', 'VB', 'NN', 'TO', 'RB', 'VB', 'PRP', 'VBD', 'IN', 'CD', 'NNS', ':', 'DT', 'VB', 'VBP', 'DT', 'NN', 'CC', 'DT', 'VB', 'VBP', 'RB', ',', "''", 'DT', 'NN', 'NN', 'NN', 'VBD', 'IN', 'DT', 'JJ', 'NN', '.']

------------TEST CASE  1719  ------------------
Input x :  ['The', 'report', 'takes', 'a', 'more', 'alarmed', 'view', 'of', 'AIDS', 'and', 'recommends', 'a', 'more', 'sweeping', 'response', 'than', 'many', 'other', 'analyses', '.']
Original Output y :  ['DT', 'NN', 'VBZ', 'DT', 'RBR', 'VBN', 'NN', 'IN', 'NNP', 'CC', 'VBZ', 'DT', 'JJR', 'VBG', 'NN', 'IN', 'JJ', 'JJ', 'NNS', '.']
Predicted Output y_pred :  ['DT', 'NN', 'VBZ', 'DT', 'VB', 'IN', 'NN', 'IN', 'DT', 'CC', 'VBZ', 'DT', 'VB', 'VBG', 'NN', 'IN', 'JJ', 'JJ', 'NN', '.']

------------TEST CASE  1720  ------------------
Input x :  ['It', 'warns', 'that', 'the', 'AIDS', 'epidemic', '``', 'may', 'reduce', 'the', 'rate', 'of', 'growth', 'of', '

Predicted Output y_pred :  ['NNS', 'VBD', 'IN', 'DT', 'JJ', 'NN', 'NN', 'POS', 'JJ', 'JJ', 'NN', 'TO', 'DT', 'NNP', 'NN', 'NNS', 'VBD', 'IN', 'RB', 'IN', 'DT', '``', 'DT', 'NN', 'IN', 'DT', 'NNS', '.', "''"]

------------TEST CASE  1732  ------------------
Input x :  ['The', 'U.S.', 'Commerce', 'Department', 'reported', 'a', '$', '10.77', 'billion', 'deficit', 'in', 'August', ',', 'compared', 'with', 'a', 'revised', 'July', 'deficit', 'of', '$', '8.24', 'billion', '.']
Original Output y :  ['DT', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', '$', 'CD', 'CD', 'NN', 'IN', 'NNP', ',', 'VBN', 'IN', 'DT', 'VBN', 'NNP', 'NN', 'IN', '$', 'CD', 'CD', '.']
Predicted Output y_pred :  ['DT', 'NNP', 'NNP', 'NNP', 'VBD', 'DT', 'VB', 'CD', 'CD', 'NN', 'IN', 'NNP', ',', 'VBN', 'IN', 'DT', 'VBN', 'NNP', 'NN', 'IN', 'VB', 'CD', 'CD', '.']

------------TEST CASE  1733  ------------------
Input x :  ['Economists', 'had', 'expected', 'a', '$', '9.1', 'billion', 'gap', '.']
Original Output y :  ['NNS', 'VBD', 'VBN', '

Predicted Output y_pred :  ['DT', 'NN', 'IN', 'NNP', 'NNP', ',', 'DT', 'NNP', ',', 'NNP', ',', 'DT', 'NNP', ',', 'NNP', ',', 'NNP', 'CC', 'NNP', '.']

------------TEST CASE  1746  ------------------
Input x :  ['The', 'so-called', 'Louvre', 'accord', 'was', 'seen', 'to', 'have', 'set', 'ranges', 'of', '1.70', 'marks', 'to', '1.90', 'marks', 'and', '120', 'yen', 'to', '140', 'yen', '.']
Original Output y :  ['DT', 'NNP', 'NNP', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'VBN', 'NNS', 'IN', 'CD', 'NNS', 'TO', 'CD', 'NNS', 'CC', 'CD', 'NN', 'TO', 'CD', 'NN', '.']
Predicted Output y_pred :  ['DT', 'JJ', 'NN', 'NN', 'VBD', 'VBN', 'TO', 'VB', 'VBN', 'NNS', 'IN', 'CD', 'NNS', 'TO', 'CD', 'NNS', 'CC', 'CD', 'NN', 'TO', 'CD', 'NN', '.']

------------TEST CASE  1747  ------------------
Input x :  ['They', 'say', 'that', 'the', 'recent', 'injection', 'of', 'liquidity', 'into', 'the', 'U.S.', 'banking', 'system', 'has', 'been', 'modest', ',', 'and', 'they', 'do', "n't", 'anticipate', 'significant', 'easing',

Predicted Output y_pred :  ['DT', 'NNS', 'VBP', 'JJ', 'NN', 'VBN', 'NNS', 'IN', 'VBP', 'NNS', 'IN', 'DT', 'NN', 'NNS', ',', 'NN', 'CC', 'JJ', 'NN', 'NNS', '.']

------------TEST CASE  1760  ------------------
Input x :  ['Linear', 'Technology', ',', 'Milpitas', ',', 'Calif.', ',', 'called', 'the', 'settlement', '``', 'positive', ',', "''", 'since', 'products', 'covered', 'by', 'the', 'disputed', 'patents', 'account', 'for', 'about', '20', '%', 'of', 'its', 'annual', 'sales', '.']
Original Output y :  ['NNP', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'VBD', 'DT', 'NN', '``', 'JJ', ',', "''", 'IN', 'NNS', 'VBN', 'IN', 'DT', 'VBN', 'NNS', 'VBP', 'IN', 'IN', 'CD', 'NN', 'IN', 'PRP$', 'JJ', 'NNS', '.']
Predicted Output y_pred :  ['NNP', 'NNP', ',', 'NNP', ',', 'NNP', ',', 'VBD', 'DT', 'NN', '``', 'JJ', ',', "''", 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'NNS', 'VBP', 'IN', 'IN', 'CD', 'NN', 'IN', 'VB', 'JJ', 'NNS', '.']

------------TEST CASE  1761  ------------------
Input x :  ['The', 'electronics